In [1]:
from transformers import AutoConfig, AutoTokenizer, AutoModel
from data.acronymDataset import AcronymDataset

/opt/anaconda3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [60]:
model_name = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=config.max_position_embeddings)
pre_trained_model = AutoModel.from_pretrained(model_name)

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Test with the data

In [67]:
%load_ext autoreload
%autoreload 2
file_path = 'data/acronym_data.txt'
dataset = AcronymDataset(file_path=file_path, tokenizer=tokenizer)
data = dataset.data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[INFO] Dataset already been loaded, using the cached dataset..


In [69]:
dataset.preprocss_dataset()
data_tokens = dataset.preprocessed_dataset

In [74]:
train_loader, val_loader = dataset.get_dataloaders(train_size=0.9, batch_size=16)

In [75]:
batch = next(iter(train_loader))

TypeError: 'int' object is not iterable

In [5]:
from models.multiHeadModel import MultiHeadModel
from models.heads import ClassificationHead

In [6]:
in_features = config.hidden_size
two_labels_head = ClassificationHead(in_features=in_features, out_features=2)
four_labels_head = ClassificationHead(in_features=in_features, out_features=4)

classifiers = [two_labels_head, four_labels_head]


In [7]:
multi_head_model = MultiHeadModel(pre_trained_model, classifiers)

In [8]:

output = multi_head_model(data_tokens[5000], 0)
output

tensor([[0.5054, 0.4982]], grad_fn=<SigmoidBackward0>)

In [9]:
output = multi_head_model(data_tokens[1], 1)
output

tensor([[0.5332, 0.4287, 0.5225, 0.5521]], grad_fn=<SigmoidBackward0>)